## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cherskiy,RU,68.7500,161.3000,36.09,100,92,13.31,overcast clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,37.51,87,40,5.75,scattered clouds
2,2,Vila Velha,BR,-20.3297,-40.2925,75.15,78,40,10.36,scattered clouds
3,3,Chuy,UY,-33.6971,-53.4616,50.50,83,25,2.66,scattered clouds
4,4,Vostok,RU,46.4856,135.8833,53.20,47,100,6.60,overcast clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Vila Velha,BR,-20.3297,-40.2925,75.15,78,40,10.36,scattered clouds
10,10,Vaini,TO,-21.2000,-175.2000,75.36,69,75,12.66,broken clouds
13,13,Kapaa,US,22.0752,-159.3190,82.38,85,75,19.57,broken clouds
18,18,Atuona,PF,-9.8000,-139.0333,78.89,78,21,23.94,few clouds
22,22,Saint-Philippe,RE,-21.3585,55.7679,75.11,68,3,8.21,clear sky
24,24,Port Moresby,PG,-9.4431,147.1797,83.62,79,20,8.05,few clouds
29,29,Rikitea,PF,-23.1203,-134.9692,77.20,69,10,19.98,clear sky
30,30,Nieuw Amsterdam,SR,5.8833,-55.0833,79.02,88,46,9.71,scattered clouds
35,35,Ixtapa,MX,20.7000,-105.2000,80.22,69,40,2.30,scattered clouds
50,50,Manaus,BR,-3.1019,-60.0250,81.09,78,0,6.91,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                132
City                   132
Country                132
Lat                    132
Lng                    132
Max Temp               132
Humidity               132
Cloudiness             132
Wind Speed             132
Current Description    132
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vila Velha,BR,75.15,scattered clouds,-20.3297,-40.2925,
10,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,
13,Kapaa,US,82.38,broken clouds,22.0752,-159.3190,
18,Atuona,PF,78.89,few clouds,-9.8000,-139.0333,
22,Saint-Philippe,RE,75.11,clear sky,-21.3585,55.7679,
24,Port Moresby,PG,83.62,few clouds,-9.4431,147.1797,
29,Rikitea,PF,77.20,clear sky,-23.1203,-134.9692,
30,Nieuw Amsterdam,SR,79.02,scattered clouds,5.8833,-55.0833,
35,Ixtapa,MX,80.22,scattered clouds,20.7000,-105.2000,
50,Manaus,BR,81.09,clear sky,-3.1019,-60.0250,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   132
Country                132
Max Temp               132
Current Description    132
Lat                    132
Lng                    132
Hotel Name             132
dtype: int64

In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))